### Установка необходимых пакетов
Читаем из файла `requirements.txt`

**Библиотеки:**
* PyTorch v2.1.0
* Sentencepiece v0.1.99
* Transformers v4.34.1
* Pandas  v2.1.1
* NumPy v1.26.1
* Scikit Learn v1.3.2


In [9]:
%pip install -r requirements.txt


[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Считываем датасет

Файлы берём из папки `/dataset`

Для обучения - `train.csv`

Для валидации - `validate.csv`

Для тестирования - `test.csv`

In [10]:
import pandas as pd # Импортируем библиотеку Pandas

# Отключаем предупреждения
no_deprecation_warning=True

In [11]:
train_data = pd.read_csv('./dataset/train.csv')     # Считываем данные для обучения
valid_data = pd.read_csv('./dataset/validate.csv')  # Считываем данные для валидации

### Инициализируем библиотеку BERT (`rubert-tiny`)

Импортируем необходимые модули

In [12]:
from bert_classifier import BertClassifier  # Импортируем основной класс BertClassifier

Инициализируем `BertClassifier` на `41` группу и `60` эпох

In [13]:
TORCH_USE_CUDA_DSA=True

classifier = BertClassifier(
    model_path='cointegrated/rubert-tiny',          # Путь до репозитория с дообучаемой моделью
    tokenizer_path='cointegrated/rubert-tiny',      # Путь до токенизатора
    n_classes=45,                                   # Количество классов для обучения
    epochs=20,                                      # Количество эпох обучения
    max_len=512,                                   # Максимальный размер текста
    model_save_path='./output/model.pt'             # Путь до сохранения модели
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Подготавливаем данные для обучения и валидации

In [14]:
classifier.preparation(
    X_train=list(train_data['groups']),             # Обучающие поля таблицы с текстом
    y_train=list(train_data['code']),               # Обучающие поля таблицы с реальными кодами групп
    X_valid=list(valid_data['groups']),             # Валидирующие поля таблицы с текстом
    y_valid=list(valid_data['code'])                # Валидирующие поля таблицы с реальными кодами групп
)

c:\Users\mrdas\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Проверяем возможность обучения нейросети на CUDA ядрах и выводим дополнительную информацию

In [15]:
import torch
print(f"CUDA поддерживается этой системой: {torch.cuda.is_available()}")
print(f"Версия CUDA: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID текущего CUDA устройства: {torch.cuda.current_device()}")
       
print(f"Имя текущего CUDA устройства: {torch.cuda.get_device_name(cuda_id)}")

CUDA поддерживается этой системой: True
Версия CUDA: 11.8
ID текущего CUDA устройства: 0
Имя текущего CUDA устройства: NVIDIA GeForce RTX 4070 Ti


### Обучаем нейросеть

In [16]:
classifier.train()

Эпоха 1/20
Обучение: потери 3.5979563163144745 | точность 0.09036969420356002
Валидация: потери 3.5749213993549347 | точность 0.09375
----------
Эпоха 2/20
Обучение: потери 3.230435023342606 | точность 0.2952989502510269
Валидация: потери 3.43543604016304 | точность 0.134765625
----------
Эпоха 3/20
Обучение: потери 2.9262669886985835 | точность 0.3879507074395253
Валидация: потери 3.352844499051571 | точность 0.15625
----------
Эпоха 4/20
Обучение: потери 2.694528802467959 | точность 0.4440894568690096
Валидация: потери 3.310773514211178 | точность 0.1796875
----------
Эпоха 5/20
Обучение: потери 2.5025435016103033 | точность 0.4842537654039251
Валидация: потери 3.287717193365097 | точность 0.1875
----------
Эпоха 6/20
Обучение: потери 2.347085857913442 | точность 0.5285257873117298
Валидация: потери 3.2703303024172783 | точность 0.193359375
----------
Эпоха 7/20
Обучение: потери 2.21569923240773 | точность 0.5563669557279781
Валидация: потери 3.316614791750908 | точность 0.189453125


### Тестируем работу обученной модели

In [17]:
import time

test_data  = pd.read_csv('./dataset/test.csv')

texts = list(test_data['groups'])
labels = list(test_data['code'])

predictions = [classifier.predict(t) for t in texts]

report = []     # form report variable

for t, l, p in zip(texts, labels, predictions):
    report.append({
        "real_group": l,
        "predicted_group": p['prediction'],
        "isEqual": ("no", "yes")[l == p['prediction']],
        "groups_set": t
    })

# Экспортируем результаты в CSV файл, поддерживаемый Excel
report_df = pd.DataFrame(report)
report_df.to_csv(f"./reports/report{time.time()}.csv", index_label="id", sep=";", encoding="cp1251")

print(f"Точность предсказаний: {(report_df[(report_df['isEqual']=='yes')].count()['isEqual']/report_df.__len__())}")

Точность предсказаний: 0.8497652582159625


C:\Users\mrdas\AppData\Local\Temp\ipykernel_7900\226904329.py:16: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  "isEqual": ("no", "yes")[l == p['prediction']],


Выводим таблицу с результатами для сверки

In [18]:
report_df.head()
report_df.to_csv(f"./reports/train_report{time.time()}.csv", index_label="id", sep=";", encoding="cp1251")

,real_group,predicted_group,isEqual,groups_set
0,1,1,yes,ежик в матане околошкольная математика борис т...
1,1,1,yes,physicsmathcode высшая математика – просто и д...
2,1,1,yes,physicsmathcode справедливые мемы про матешу м...
3,1,1,yes,околошкольная математика борис трушин physicsm...
4,1,1,yes,типичный математик code blog physicsmathcode ч...


### Получаем метрики

In [19]:
from sklearn.metrics import precision_recall_fscore_support

predictions_all = [p['prediction'] for p in predictions]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions_all, average='macro', zero_division=0)[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7634548611111112, recall: 0.8040178571428571, f1score: 0.7715719567741626
